# BERT Model for summarization for CASS French Dataset


# Example Legal Document in French

In [15]:
with open('sampletestcases/JURITEXT000007000221.txt') as f:
    case_text = f.read()

In [16]:
case_text

"sur le moyen unique : attendu que la caisse primaire fait grief a la cour d' appel d' avoir dit que dame streelecki proprietaire indivise avec ses enfants d' un fonds de blanchisserie - laverie qu' elle avait concede a compter du 1er avril 1972 en location - gerance a son frere avait ete a bon droit , affiliee par celui - ci au regime general de la securite sociale au titre de l' activite de repasseuse salariee qu' elle avait exercee dans le fonds a partir du 2 mai 1972 alors que la situation de proprietaire indivis du fonds de commerce etait exclusive d' un etat de subordination de dame x... vis - a - vis du locataire gerant , son frere ;\nmais attendu que la cour d' appel releve qu' il n' est pas conteste que , sans fraude , dame x... a donne en location - gerance a compter du 1er avril 1972 le fonds de commerce dont elle etait coproprietaire indivise ;\nqu' elle est entree le 2 mai 1972 au service du locataire gerant en executant , sans aucun pouvoir de direction sur la marche de l

In [21]:
#Running code for a set of 20 legal documents 
#sampletestcases is the folder which contains 20 legal documents

In [22]:
import os
from sentence_transformers import SentenceTransformer
from rouge import Rouge
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min

def evaluate(model_sum, gt_sum):
    rouge = Rouge()
    return rouge.get_scores(model_sum, gt_sum, avg=True)



input_path_cases= os.listdir('sampletestcases/')
model_sum = []
true_sum = []
embedder = SentenceTransformer('distiluse-base-multilingual-cased')
root = "sampletestcases"
for file in input_path_cases:
    file_path = os.path.join(root,file)
    if '.txt' in file_path:
        with open(file_path) as f:
            corpus = f.read()
        res = corpus.split("@summary ")
        text,corpus_summary = res[0],res[1]
        legal_corpus = text.split(" , ")
        sentence_embeddings = embedder.encode(legal_corpus)
        enc_embedding = sentence_embeddings
        n_clusters = int(np.ceil(len(enc_embedding)**0.5))
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans = kmeans.fit(enc_embedding)
        avg = []
        closest = []
        for j in range(n_clusters):
            idx = np.where(kmeans.labels_ == j)[0]
            avg.append(np.mean(idx))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                   enc_embedding)
        ordering = sorted(range(n_clusters), key=lambda k: avg[k])
        gen_summary = ' '.join([legal_corpus[closest[idx]] for idx in ordering])
        model_sum.append(gen_summary)
        true_sum.append(corpus_summary)


print("ROUGE score: {}".format(evaluate(model_sum, true_sum)))

ROUGE score: {'rouge-1': {'f': 0.39499565914606827, 'p': 0.41642887606506196, 'r': 0.42123943076273473}, 'rouge-2': {'f': 0.17382500770742584, 'p': 0.18503580633895717, 'r': 0.1833161097304338}, 'rouge-l': {'f': 0.26467613788232125, 'p': 0.27126577515267214, 'r': 0.2776789134404475}}


In [24]:
model_sum[5]

"leurs activites identiques ou complementaires alors que les deux societes relevant d' inspecteurs du travail differents ( celui du port autonome pour l' une et celui des transports pour l' autre ) pris de la violation des articles l. 412 et suivants du code du travail : attendu qu' il est reproche au jugement attaque d' avoir decide que les deux societes korsia - revel et france - afrique formaient un ensemble economique et social unique permettant la designation d' un delegue syndical commun enfin que ne faisait pas obstacle a cette unite le fait qu' elles fussent soumises a des conventions collectives differentes et ne relevassent pas du meme inspecteur du travail vu la connexite"

In [25]:
true_sum[5]

"constituent un ensemble economique et social , justifiant la designation d' un delegue syndical commun , deux societes dont les activites sont tellement similaires que le personnel d' encadrement est commun et le personnel d' execution interchangeable , l' une d' entre elles exercant simplement une activite supplementaire de caractere accessoire , et qui ne se differencient que par des elements secondaires ( controle medical et inspection du travail ) . "

In [23]:
model_sum

["l. 433 - 2 du code du travail et 455 du nouveau code de procedure civile ;\nattendu que les elections pour le renouvellement des membres des comites d' etablissement institues dans la bank of america l' autre groupant la succursale de paris et les agences de lyon que dans l' agence de strasbourg avait eu des elus aux elections precedentes ;\nd' ou il suit que le tribunal a faussement applique et donc viole les textes susvises ;\npar ces motifs : casse et annule le jugement rendu entre les parties le 28 juin 1977 par le tribunal d' instance du 16e arrondissement de paris ;\nremet cependant",
 "affiliee par celui - ci au regime general de la securite sociale au titre de l' activite de repasseuse salariee qu' elle avait exercee dans le fonds a partir du 2 mai 1972 alors que la situation de proprietaire indivis du fonds de commerce etait exclusive d' un etat de subordination de dame x... vis - a - vis du locataire gerant sans aucun pouvoir de direction sur la marche de l' entreprise un t